In [1]:
from colorama import init, Fore
import logging
import sys
import datetime
from dateutil.parser import parse
import re


In [ ]:

init(autoreset=True)

log_level = logging.INFO # logging.DEBUG


In [ ]:

logging.basicConfig(
    stream=sys.stdout,
    level=log_level,
    format='[%(levelname)s] %(message)s'
)


In [2]:

def extract_units(fetch_log):
    units = set()
    with open(fetch_log,'r') as f:
        while True:
            line = f.readline()
            if line.startswith("[INFO] Started to"):
                pass
            else:
                break
            p = re.compile("[A-Z0-9]+/([A-Z0-9]+)")
            patterns =p.search(line)
            unit = patterns.groups()[0]
            units.add(unit)
    return units


def extract_repair_log(target):
    with open(target) as f:
        log_data=f.readlines()
    # lo    
    log_list = []
    for i in range(len(log_data)):
        if '\n' == log_data[i]:
            pass
        if 'Loading' in log_data[i]:
            log_list.append(''.join(log_data[i:i+7]))

    log_dict = dict()
    for log in log_list:
        key=log.split('\n')[0].split("/")[1].split("_")[1]
        log_dict[key] = log[:-1]
    return log_dict


In [ ]:

def write_toml(symbol, since):
    until = "2022-07-01 00:00:00"

    toml_str="""[settings]

    debug = false
    persistence = "file"
    append_file = true
    proxies = [
        ""
    ]
    ratelimit = true

    [exchanges]

        [exchanges.binance]
        filter_symbols = ["{0}"]
        filter_resolutions = ["1h"]
        since = "{1}"
        until = "{2}"
        limit = 365""".format(symbol, since, until)

    path = "../toml/"
    file = "{}+{}.toml".format(symbol.replace('/',''),since).replace(' ','_')

    with open(path+file, "w") as f:
        f.write(toml_str)
    return path+file
    


In [3]:
def make_symbol_format(symbolwo,units):
    for unit in units:
        if symbolwo.endswith(unit):
            return "%s/%s"%(symbolwo[:-len(unit)],unit)
    return


In [6]:

def main():
    cnt = 0
    ms_cnt = 0
    ms_symbol = []
    units = extract_units("../log/data_entire_fetch.log" )
    logging.debug("%dec units are collected"%len(units))
    
    target="../log/data_entire_repair_2.log"
    log_dict = extract_repair_log(target)
    
    start_point =  "Date of first data point 2020-07-01 00:00:00"
    last_point = "2022-07-01 00:00:00"
    missing =  "Found 0 missing data points (0.0% of the data is missing)"
    good=[]
    for k,v in log_dict.items():
        status=0
        symbol = make_symbol_format(k,units)
        if not symbol:
            logging.error("Cannot change symbol name")
            continue
        logging.info("Symbol: %s"%symbol)
        lines = v.split('\n')
        
        if start_point in lines[2]:
            logging.debug("Good Start")
            status+=1
        else:
            logging.debug("Late Start")

        if missing in lines[5]:
            logging.debug("Pass Missing")
            status+=1
        else:
            logging.debug("Missing Data")
            ms_cnt += 1
            ms_symbol.append(symbol)
            continue

        last_str=" ".join(v.split("\n")[3].split()[5:])
        if parse(last_str) < parse(last_point):
            # logging.warn("Lack data")
            # file_name = write_toml(symbol,last_str)
            # cnt+=1
            # logging.info("Create toml file to re-fetch data %s"%file_name,'-'*10)
            logging.debug("Lack data")
        else:
            logging.debug("Full data")
            status+=1
        if status ==3:
            good.append(symbol)
        logging.info("-"*80)

    # if ms_symbol:
    #     with open("mssing_symbol.txt",'w') as f:
    #         for ms in ms_symbol:
    #             f.write("%s\n"%ms)

        
    data_ec  = int(lines[-1].split()[0])
    logging.info("Data count: %d & mssing data: %d"%(data_ec,ms_cnt))
    return good
if __name__ == '__main__':
    good=main()
    # logging.info("%d ec toml fils be created"%cnt)

In [8]:
len(good)

408

In [ ]:
a